In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [14]:
transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

batch_size = 100

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
"""class DenseNN(nn.Module):
    def __init__(self):
        super(DenseNN, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 2048)  
        self.fc2 = nn.Linear(2048, 1024)           
        self.fc3 = nn.Linear(1024, 512)   
        self.fc4 = nn.Linear(512, 256) 
        self.dropout = nn.Dropout(p=0.5)        
        self.fc5 = nn.Linear(256, 10)           

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)             
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)                        
        return x """

'class DenseNN(nn.Module):\n    def __init__(self):\n        super(DenseNN, self).__init__()\n        self.fc1 = nn.Linear(3 * 32 * 32, 2048)  \n        self.fc2 = nn.Linear(2048, 1024)           \n        self.fc3 = nn.Linear(1024, 512)   \n        self.fc4 = nn.Linear(512, 256) \n        self.dropout = nn.Dropout(p=0.5)        \n        self.fc5 = nn.Linear(256, 10)           \n\n    def forward(self, x):\n        x = x.view(-1, 3 * 32 * 32)             \n        x = torch.relu(self.fc1(x))\n        x = torch.relu(self.fc2(x))\n        x = torch.relu(self.fc3(x))\n        x = torch.relu(self.fc4(x))\n        x = self.dropout(x)\n        x = self.fc5(x)                        \n        return x '

In [16]:
class DenseNN(nn.Module):
    def __init__(self):
        super(DenseNN, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 1024)  
        self.fc2 = nn.Linear(1024, 512)           
        self.fc3 = nn.Linear(512, 10)                    


    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)             
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)                        
        return x

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNN().to(device)                   
criterion = nn.CrossEntropyLoss()

num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()             # Zero the parameter gradients
        outputs = model(images)           # Forward pass: Compute predictions
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()                   # Backward pass: Compute gradients
        optimizer.step()                  # Update weights
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader):.4f} Accuracy: {(correct*100) / total} %")

print("Finished Training")

Epoch 1/5, Loss: 1.6433 Accuracy: 41.722 %
Epoch 2/5, Loss: 1.4243 Accuracy: 49.794 %
Epoch 3/5, Loss: 1.3092 Accuracy: 53.602 %
Epoch 4/5, Loss: 1.2064 Accuracy: 57.162 %
Epoch 5/5, Loss: 1.1216 Accuracy: 59.952 %
Finished Training


In [18]:


model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy after epoch {epoch + 1}: {accuracy:.2f}%")

Test Accuracy after epoch 5: 53.42%
